In [1]:
#Importing Libraries
import pandas as pd
import yaml
import os
import time
import numpy as np
import warnings


In [2]:
warnings.filterwarnings("ignore")  # Hides messy Numpy warnings

# function for extracting match details from yaml files
def extract_details(filename):
    dict = yaml.safe_load(open(filename)) #Read yaml files
    flag = False
    
    match_details = {}
    match_details['venue'] = dict['info']['venue']
    match_details['date'] = dict['info']['dates'][0]
    match_details['team1'] = dict['info']['teams'][0]
    match_details['team2'] = dict['info']['teams'][1]
    
    # extracts winner of the match from the outcome section
    if 'winner' not in dict['info']['outcome']:
        match_details['winner'] = 'no_result'
    else:
        match_details['winner'] = dict['info']['outcome']['winner']
    
    # extracts who wins the toss for assigning currently playing team and its opposition team
    if dict['info']['toss']['winner'] == match_details['team1']:
        if dict['info']['toss']['decision'] == 'bat':
            match_details['bat_first'] = [match_details['team1'],match_details['team2']]
        else:
            match_details['bat_first'] = [match_details['team2'],match_details['team1']]
    else:
        if dict['info']['toss']['decision'] == 'bat':
            match_details['bat_first'] = [match_details['team2'],match_details['team1']]
        else:
            match_details['bat_first'] = [match_details['team1'],match_details['team2']]
    print(match_details)
    
     # To store details about each player's performance in a specific cricket match.

    player_details = pd.DataFrame(columns=['date','team','runs','balls','not_out','opposition','batting_innings','venue','outcome'])
    
    innings1 = dict['innings'][0]['1st innings']['deliveries']
    
    for ball in innings1:
        for delivery in ball:
      
            try: # to check if the batsman is already in the dataframe 
                player_details.index.get_loc(ball[delivery]['batsman'])
            except: # if the batsman is not in the dataframe then it assigns his runs and balls to 0
                player_details.loc[ball[delivery]['batsman'],'runs'] = 0
                player_details.loc[ball[delivery]['batsman'], 'balls'] = 0
                
            #Information about the current batsman's performance, including the date, team, opposition, batting innings, and venue
            
            player_details.loc[ball[delivery]['batsman'], 'date'] = match_details['date']
            player_details.loc[ball[delivery]['batsman'], 'team'] = match_details['bat_first'][0]
            player_details.loc[ball[delivery]['batsman'], 'opposition'] = match_details['bat_first'][1]
            player_details.loc[ball[delivery]['batsman'], 'batting_innings'] = 1
            player_details.loc[ball[delivery]['batsman'], 'venue'] = match_details['venue']
            
            # assigns outcome to 1 if the team for which the batsman is playing won the match
            if match_details['winner'] == player_details.loc[ball[delivery]['batsman'], 'team']:
                player_details.loc[ball[delivery]['batsman'], 'outcome'] = 1
            else:
                player_details.loc[ball[delivery]['batsman'], 'outcome'] = 0
                
            # this if statement updates if the batsman got out
            if 'wicket' in ball[delivery]:
                player_details.loc[ball[delivery]['batsman'],'not_out'] = False
            else:
                player_details.loc[ball[delivery]['batsman'], 'not_out'] = True
                
            # update the runs and balls columns for the batsman in the player_details
            
            player_details.loc[ball[delivery]['batsman'],'runs'] += ball[delivery]['runs']['batsman']
            player_details.loc[ball[delivery]['batsman'],'balls'] += 1
        
    # The below code extracts the player details of the match if there is a second innings and the rest of the logic is same as 1st
    try:
        innings2 = dict['innings'][1]['2nd innings']['deliveries']
    except:
        flag = True # sets flag to true if there is no 2nd innings
    else:
        for ball in innings2:
            for delivery in ball:
            
                try:
                    player_details.index.get_loc(ball[delivery]['batsman'])
                except:
                    player_details.loc[ball[delivery]['batsman'],'runs'] = 0
                    player_details.loc[ball[delivery]['batsman'], 'balls'] = 0

                player_details.loc[ball[delivery]['batsman'], 'date'] = match_details['date']
                player_details.loc[ball[delivery]['batsman'], 'team'] = match_details['bat_first'][1]
                player_details.loc[ball[delivery]['batsman'], 'opposition'] = match_details['bat_first'][0]
                player_details.loc[ball[delivery]['batsman'], 'batting_innings'] = 2
                player_details.loc[ball[delivery]['batsman'], 'venue'] = match_details['venue']
                
                if match_details['winner'] == player_details.loc[ball[delivery]['batsman'], 'team']:
                    player_details.loc[ball[delivery]['batsman'], 'outcome'] = 1
                else:
                    player_details.loc[ball[delivery]['batsman'], 'outcome'] = 0
                    
                if 'wicket' in ball[delivery]:
                    player_details.loc[ball[delivery]['batsman'],'not_out'] = False
                else:
                    player_details.loc[ball[delivery]['batsman'], 'not_out'] = True

                player_details.loc[ball[delivery]['batsman'],'runs'] += ball[delivery]['runs']['batsman']
                player_details.loc[ball[delivery]['batsman'],'balls'] += 1                

    return player_details

In [3]:
# Extracting overall batsman details and batsman details in every single match
overall_batsman_details = pd.DataFrame(columns=['team','innings','runs','balls', 'average', 'strike_rate','centuries','fifties','zeros'])
match_batsman_details = pd.DataFrame(columns=['date','name','team','opposition','venue','innings_played','previous_average','previous_strike_rate','previous_centuries','previous_fifties','previous_zeros','runs','balls'])

count = -1

In [4]:
# iterates through all files in the path
for filename in os.listdir('./../new_odis'):
    if filename.endswith('.yaml'): # To check if the file name ends with .yaml
        
        player_details = extract_details('./../new_odis/'+filename) # calls the extract_details function to get the player details from a specific match

        for player in player_details.index:
            count += 1
            try:
                overall_batsman_details.index.get_loc(player) #check if the player details are already exsist in the overall batsman details or not
            except: # if the player is not in the overall batsman details if adds the player and initialize some parameters
                overall_batsman_details.loc[player,'team'] = player_details.loc[player,'team']
                overall_batsman_details.loc[player,'innings'] = 0
                overall_batsman_details.loc[player,'runs'] = 0
                overall_batsman_details.loc[player,'balls'] = 0
                overall_batsman_details.loc[player,'strike_rate'] = 0
                overall_batsman_details.loc[player, 'average'] = 0
                overall_batsman_details.loc[player,'zeros'] = 0
                overall_batsman_details.loc[player,'fifties'] = 0
                overall_batsman_details.loc[player,'centuries'] = 0

            
            # updates bastman details in each match
            match_batsman_details.loc[count,'date'] = player_details.loc[player,'date']
            match_batsman_details.loc[count,'name'] = player
            match_batsman_details.loc[count,'team'] = player_details.loc[player,'team']
            match_batsman_details.loc[count,'opposition'] = player_details.loc[player,'opposition']
            match_batsman_details.loc[count,'venue'] = player_details.loc[player,'venue']
            match_batsman_details.loc[count,'innings_played'] = overall_batsman_details.loc[player,'innings']
            match_batsman_details.loc[count,'previous_average'] = overall_batsman_details.loc[player, 'average']
            match_batsman_details.loc[count,'previous_strike_rate'] = overall_batsman_details.loc[player, 'strike_rate']
            match_batsman_details.loc[count,'previous_centuries'] = overall_batsman_details.loc[player,'centuries']
            match_batsman_details.loc[count,'previous_fifties'] = overall_batsman_details.loc[player,'fifties']
            match_batsman_details.loc[count,'previous_zeros'] = overall_batsman_details.loc[player,'zeros']
            match_batsman_details.loc[count,'runs'] = player_details.loc[player,'runs']
            match_batsman_details.loc[count,'balls'] = player_details.loc[player,'balls']
            
            #updates overall batsman details
            overall_batsman_details.loc[player,'innings'] += 1
            overall_batsman_details.loc[player,'runs'] += player_details.loc[player,'runs']
            overall_batsman_details.loc[player,'balls'] += player_details.loc[player,'balls']
            overall_batsman_details.loc[player,'strike_rate'] = (((player_details.loc[player, 'runs']/player_details.loc[player, 'balls'])*100)+(overall_batsman_details.loc[player,'strike_rate']*(overall_batsman_details.loc[player,'innings']-1)))/overall_batsman_details.loc[player,'innings']
            overall_batsman_details.loc[player,'average'] = ((player_details.loc[player, 'runs'])+(overall_batsman_details.loc[player,'average']*(overall_batsman_details.loc[player,'innings']-1)))/overall_batsman_details.loc[player,'innings']
            
            # updating each player half century, century and zeroes count
            if player_details.loc[player,'runs'] == 0:
                overall_batsman_details.loc[player,'zeros'] += 1
            elif player_details.loc[player,'runs']>=50 and player_details.loc[player,'runs']<100:
                overall_batsman_details.loc[player,'fifties'] += 1
            elif player_details.loc[player,'runs']>=100:
                overall_batsman_details.loc[player,'centuries'] += 1

{'venue': 'Takashinga Sports Club, Highfield, Harare', 'date': '2023-07-02', 'team1': 'United Arab Emirates', 'team2': 'Nepal', 'winner': 'Nepal', 'bat_first': ['United Arab Emirates', 'Nepal']}
{'venue': 'Kingsmead', 'date': '2005-02-11', 'team1': 'South Africa', 'team2': 'England', 'winner': 'no_result', 'bat_first': ['South Africa', 'England']}
{'venue': 'Rawalpindi Cricket Stadium', 'date': '2004-03-16', 'team1': 'Pakistan', 'team2': 'India', 'winner': 'Pakistan', 'bat_first': ['Pakistan', 'India']}
{'venue': 'Senwes Park', 'date': '2019-05-09', 'team1': 'South Africa', 'team2': 'Pakistan', 'winner': 'South Africa', 'bat_first': ['Pakistan', 'South Africa']}
{'venue': 'Mahinda Rajapaksa International Cricket Stadium, Sooriyawewa', 'date': datetime.date(2012, 11, 10), 'team1': 'Sri Lanka', 'team2': 'New Zealand', 'winner': 'Sri Lanka', 'bat_first': ['New Zealand', 'Sri Lanka']}
{'venue': 'Wanderers Cricket Ground, Windhoek', 'date': '2021-11-26', 'team1': 'Namibia', 'team2': 'Oman',

{'venue': 'Sir Vivian Richards Stadium, North Sound', 'date': datetime.date(2017, 3, 5), 'team1': 'West Indies', 'team2': 'England', 'winner': 'England', 'bat_first': ['West Indies', 'England']}
{'venue': 'Civil Service Cricket Club, Stormont', 'date': '2018-08-31', 'team1': 'Ireland', 'team2': 'Afghanistan', 'winner': 'Afghanistan', 'bat_first': ['Ireland', 'Afghanistan']}
{'venue': 'Al Amerat Cricket Ground Oman Cricket (Ministry Turf 1)', 'date': '2021-09-19', 'team1': 'Oman', 'team2': 'Nepal', 'winner': 'Nepal', 'bat_first': ['Oman', 'Nepal']}
{'venue': 'Manuka Oval', 'date': datetime.date(2016, 11, 20), 'team1': 'Australia', 'team2': 'South Africa', 'winner': 'Australia', 'bat_first': ['Australia', 'South Africa']}
{'venue': 'ICC Academy', 'date': '2019-01-25', 'team1': 'United Arab Emirates', 'team2': 'Nepal', 'winner': 'United Arab Emirates', 'bat_first': ['Nepal', 'United Arab Emirates']}
{'venue': 'Brisbane Cricket Ground, Woolloongabba', 'date': datetime.date(2015, 1, 20), 't

{'venue': 'Vidarbha Cricket Association Stadium, Jamtha', 'date': datetime.date(2011, 3, 12), 'team1': 'India', 'team2': 'South Africa', 'winner': 'South Africa', 'bat_first': ['India', 'South Africa']}
{'venue': 'ICC Global Cricket Academy', 'date': datetime.date(2017, 3, 2), 'team1': 'United Arab Emirates', 'team2': 'Ireland', 'winner': 'Ireland', 'bat_first': ['Ireland', 'United Arab Emirates']}
{'venue': 'Basin Reserve, Wellington', 'date': '2022-03-24', 'team1': 'South Africa', 'team2': 'West Indies', 'winner': 'no_result', 'bat_first': ['South Africa', 'West Indies']}
{'venue': 'Eden Park', 'date': '2004-01-03', 'team1': 'Pakistan', 'team2': 'New Zealand', 'winner': 'New Zealand', 'bat_first': ['Pakistan', 'New Zealand']}
{'venue': 'Docklands Stadium', 'date': datetime.date(2006, 2, 3), 'team1': 'Australia', 'team2': 'South Africa', 'winner': 'Australia', 'bat_first': ['Australia', 'South Africa']}
{'venue': 'County Ground', 'date': datetime.date(2017, 7, 8), 'team1': 'New Zealan

{'venue': 'Kensington Oval, Bridgetown', 'date': datetime.date(2011, 5, 2), 'team1': 'West Indies', 'team2': 'Pakistan', 'winner': 'West Indies', 'bat_first': ['Pakistan', 'West Indies']}
{'venue': 'Kinrara Academy Oval', 'date': datetime.date(2006, 9, 16), 'team1': 'Australia', 'team2': 'India', 'winner': 'no_result', 'bat_first': ['Australia', 'India']}
{'venue': 'Trent Bridge', 'date': datetime.date(2010, 7, 8), 'team1': 'England', 'team2': 'Bangladesh', 'winner': 'England', 'bat_first': ['Bangladesh', 'England']}
{'venue': 'Sardar Patel Stadium, Motera', 'date': datetime.date(2011, 3, 24), 'team1': 'India', 'team2': 'Australia', 'winner': 'India', 'bat_first': ['Australia', 'India']}
{'venue': 'Bay Oval, Mount Maunganui', 'date': '2021-04-10', 'team1': 'Australia', 'team2': 'New Zealand', 'winner': 'Australia', 'bat_first': ['Australia', 'New Zealand']}
{'venue': 'Hazelaarweg, Rotterdam', 'date': '2021-05-19', 'team1': 'Netherlands', 'team2': 'Scotland', 'winner': 'Netherlands', 'b

{'venue': 'Kingsmead', 'date': datetime.date(2015, 8, 26), 'team1': 'South Africa', 'team2': 'New Zealand', 'winner': 'South Africa', 'bat_first': ['South Africa', 'New Zealand']}
{'venue': 'Maharashtra Cricket Association Stadium', 'date': '2021-03-26', 'team1': 'India', 'team2': 'England', 'winner': 'England', 'bat_first': ['India', 'England']}
{'venue': 'The Wanderers Stadium', 'date': '2018-02-10', 'team1': 'India', 'team2': 'South Africa', 'winner': 'South Africa', 'bat_first': ['India', 'South Africa']}
{'venue': 'MA Chidambaram Stadium, Chepauk, Chennai', 'date': '2003-10-23', 'team1': 'India', 'team2': 'New Zealand', 'winner': 'no_result', 'bat_first': ['India', 'New Zealand']}
{'venue': 'Sydney Cricket Ground', 'date': '2005-01-23', 'team1': 'Pakistan', 'team2': 'Australia', 'winner': 'Australia', 'bat_first': ['Pakistan', 'Australia']}
{'venue': 'County Ground', 'date': datetime.date(2009, 6, 29), 'team1': 'England', 'team2': 'Australia', 'winner': 'England', 'bat_first': ['A

...

{'venue': 'Castle Avenue', 'date': '2019-05-15', 'team1': 'Ireland', 'team2': 'Bangladesh', 'winner': 'Bangladesh', 'bat_first': ['Ireland', 'Bangladesh']}
{'venue': 'SuperSport Park', 'date': datetime.date(2008, 1, 20), 'team1': 'South Africa', 'team2': 'West Indies', 'winner': 'South Africa', 'bat_first': ['West Indies', 'South Africa']}
{'venue': 'Sydney Cricket Ground', 'date': datetime.date(2014, 11, 23), 'team1': 'Australia', 'team2': 'South Africa', 'winner': 'Australia', 'bat_first': ['South Africa', 'Australia']}
{'venue': 'Sir Vivian Richards Stadium, North Sound', 'date': '2019-11-01', 'team1': 'West Indies', 'team2': 'India', 'winner': 'West Indies', 'bat_first': ['West Indies', 'India']}
{'venue': 'Willowmoore Park', 'date': datetime.date(2009, 4, 1), 'team1': 'Ireland', 'team2': 'Scotland', 'winner': 'Ireland', 'bat_first': ['Scotland', 'Ireland']}
{'venue': 'Sylhet International Cricket Stadium', 'date': '2020-03-01', 'team1': 'Bangladesh', 'team2': 'Zimbabwe', 'winner':

{'venue': 'SuperSport Park', 'date': datetime.date(2013, 9, 24), 'team1': 'South Africa', 'team2': 'Bangladesh', 'winner': 'South Africa', 'bat_first': ['Bangladesh', 'South Africa']}
{'venue': 'New Wanderers Stadium, Johannesburg', 'date': '2003-03-23', 'team1': 'Australia', 'team2': 'India', 'winner': 'Australia', 'bat_first': ['Australia', 'India']}
{'venue': 'County Ground', 'date': datetime.date(2017, 7, 5), 'team1': 'India', 'team2': 'Sri Lanka', 'winner': 'India', 'bat_first': ['India', 'Sri Lanka']}
{'venue': 'Daren Sammy National Cricket Stadium, Gros Islet, St Lucia', 'date': '2023-06-28', 'team1': 'Ireland', 'team2': 'West Indies', 'winner': 'no_result', 'bat_first': ['Ireland', 'West Indies']}
{'venue': 'JSCA International Stadium Complex', 'date': datetime.date(2013, 1, 19), 'team1': 'India', 'team2': 'England', 'winner': 'India', 'bat_first': ['England', 'India']}
{'venue': 'R Premadasa Stadium, Colombo', 'date': '2021-07-23', 'team1': 'India', 'team2': 'Sri Lanka', 'winn

{'venue': 'ICC Academy', 'date': '2017-12-06', 'team1': 'Hong Kong', 'team2': 'Papua New Guinea', 'winner': 'Hong Kong', 'bat_first': ['Hong Kong', 'Papua New Guinea']}
{'venue': 'Hagley Oval', 'date': datetime.date(2015, 2, 14), 'team1': 'New Zealand', 'team2': 'Sri Lanka', 'winner': 'New Zealand', 'bat_first': ['New Zealand', 'Sri Lanka']}
{'venue': 'Eden Park Outer Oval', 'date': '2020-01-27', 'team1': 'New Zealand', 'team2': 'South Africa', 'winner': 'South Africa', 'bat_first': ['New Zealand', 'South Africa']}
{'venue': 'Clontarf Cricket Club Ground', 'date': datetime.date(2013, 5, 23), 'team1': 'Ireland', 'team2': 'Pakistan', 'winner': 'no_result', 'bat_first': ['Pakistan', 'Ireland']}
{'venue': 'Sher-e-Bangla National Cricket Stadium', 'date': '2021-05-25', 'team1': 'Bangladesh', 'team2': 'Sri Lanka', 'winner': 'Bangladesh', 'bat_first': ['Bangladesh', 'Sri Lanka']}
{'venue': 'Queens Sports Club', 'date': datetime.date(2010, 5, 30), 'team1': 'India', 'team2': 'Sri Lanka', 'winne

{'venue': 'Newlands', 'date': datetime.date(2007, 12, 2), 'team1': 'South Africa', 'team2': 'New Zealand', 'winner': 'South Africa', 'bat_first': ['New Zealand', 'South Africa']}
{'venue': 'New Wanderers Stadium', 'date': datetime.date(2013, 12, 5), 'team1': 'South Africa', 'team2': 'India', 'winner': 'South Africa', 'bat_first': ['South Africa', 'India']}
{'venue': 'New Wanderers Stadium', 'date': datetime.date(2012, 1, 22), 'team1': 'South Africa', 'team2': 'Sri Lanka', 'winner': 'Sri Lanka', 'bat_first': ['South Africa', 'Sri Lanka']}
{'venue': 'Queens Sports Club', 'date': '2018-03-15', 'team1': 'Scotland', 'team2': 'United Arab Emirates', 'winner': 'Scotland', 'bat_first': ['Scotland', 'United Arab Emirates']}
{'venue': 'Chilaw Marians Cricket Club Ground', 'date': '2018-09-16', 'team1': 'India', 'team2': 'Sri Lanka', 'winner': 'Sri Lanka', 'bat_first': ['India', 'Sri Lanka']}
{'venue': 'West End Park International Cricket Stadium, Doha', 'date': '2022-01-23', 'team1': 'Afghanista

{'venue': 'New Wanderers Stadium', 'date': datetime.date(2008, 2, 3), 'team1': 'South Africa', 'team2': 'West Indies', 'winner': 'South Africa', 'bat_first': ['West Indies', 'South Africa']}
{'venue': 'Melbourne Cricket Ground', 'date': '2003-01-25', 'team1': 'Australia', 'team2': 'England', 'winner': 'Australia', 'bat_first': ['Australia', 'England']}
{'venue': 'County Ground, Chelmsford', 'date': '2023-05-12', 'team1': 'Ireland', 'team2': 'Bangladesh', 'winner': 'Bangladesh', 'bat_first': ['Ireland', 'Bangladesh']}
{'venue': 'Jade Stadium, Christchurch', 'date': '2003-01-01', 'team1': 'India', 'team2': 'New Zealand', 'winner': 'New Zealand', 'bat_first': ['India', 'New Zealand']}
{'venue': "Lord's", 'date': datetime.date(2017, 5, 29), 'team1': 'England', 'team2': 'South Africa', 'winner': 'South Africa', 'bat_first': ['England', 'South Africa']}
{'venue': 'Al Amerat Cricket Ground Oman Cricket (Ministry Turf 1)', 'date': '2021-09-07', 'team1': 'Papua New Guinea', 'team2': 'Nepal', 'w

In [5]:

# Setting index to player_name
overall_batsman_details.index.name = 'player_name'

# Saving the excel files in Player Details folder
overall_batsman_details.to_excel('./../playerDetails/overall_batsman_details.xlsx')
match_batsman_details.to_excel('./../playerDetails/match_batsman_details.xlsx')